In [3]:
%matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
# import gmplot
# from geopy.geocoders import Nominatim
# import time
# import gmaps
# import gmaps.datasets
import folium


Using matplotlib backend: Qt5Agg


In [5]:
Aijobsindustry_dataset = pd.read_csv("AIJobsIndustry_Clean_v3.csv")
print(f"No of rows in Aijobsindustry_dataset is {Aijobsindustry_dataset.shape[0]}")

Dice_us_jobs_dataset = pd.read_csv("Dice_US_jobs_Clean_v2.csv", encoding="cp437")
print(f"No of rows in Dice_us_jobs_dataset is {Dice_us_jobs_dataset.shape[0]}")

monster_dataset = pd.read_csv("Monster_Clean.csv")
print(f"No of rows in monster_dataset is {monster_dataset.shape[0]}")

indeed_dataset = pd.read_csv("alldata_Clean_v3.csv")
print(f"No of rows in indeed_dataset is {indeed_dataset.shape[0]}")

##file needs to match format for concat
# Dice_use_Tech_filtered = pd.read_csv("Dice_US_Tech_FilteredJobData.csv")
# Dice_use_Tech_filtered

print(f"No of rows in final merged data set should be is {Aijobsindustry_dataset.shape[0]+Dice_us_jobs_dataset.shape[0]+monster_dataset.shape[0]+indeed_dataset.shape[0]}")

datasets_to_merge = [Aijobsindustry_dataset,Dice_us_jobs_dataset,monster_dataset,indeed_dataset]

##please confirm the concat makes sense
merged_dataset = pd.concat(datasets_to_merge)
print(f"Actual no of rows in merged_dataset is {merged_dataset.shape[0]}")

#merged_dataset.to_csv("Merged_Data.csv")

No of rows in Aijobsindustry_dataset is 631
No of rows in Dice_us_jobs_dataset is 1282
No of rows in monster_dataset is 162
No of rows in indeed_dataset is 6953
No of rows in final merged data set should be is 9028
Actual no of rows in merged_dataset is 9028


In [6]:
def parser(column):
    word_count = {}
    for entry in column:
        entry = re.sub('[^A-Za-z0-9 ]+', '', str(entry))
        entry = entry.lower()
#        print(entry)
        try:
            entry = entry.replace(',','')
            word_split = entry.split()
            for word in word_split:
                if word not in word_count:
                    word_count[word] = 1
                else:
                    word_count[word] += 1
        except AttributeError:
            print("Skipping an entry due to bad data")
            continue
    return word_count

In [7]:
desc = merged_dataset['Description']
value = parser(desc)
#print(value)

In [8]:
group_by_state = merged_dataset.groupby("Loc_state")
count_for_state = group_by_state.count()
#print(count_for_state)
count_for_state = count_for_state["Position"]
print(count_for_state)

Loc_state
AL       2
AR       2
AZ       7
CA    2467
CO     265
CT       4
DC     355
DE       6
FL      17
GA     295
IL     490
IN       2
KS       4
LA       4
MA    1369
MD      13
ME       1
MI       6
MN      16
MO       5
NC      33
NE       2
NH       1
NJ      85
NM       1
NV       3
NY     989
OH      40
OK       1
OR       3
PA      16
SC       6
TN       2
TX     298
VA      33
WA     970
WI       4
Name: Position, dtype: int64


In [9]:
print(value['python'])

3610


In [46]:
import pandas as pd
import folium

state_geo = os.path.join('us-states.json')
state_unemployment = os.path.join('US_Unemployment_Oct2012.csv')
state_data = pd.read_csv(state_unemployment, error_bad_lines=False)
print(state_data.head())
print(state_geo)
m = folium.Map(location=[37,-102], zoom_start=5)
m

  State  Unemployment
0    AL           7.1
1    AK           6.8
2    AZ           8.1
3    AR           7.2
4    CA          10.1
us-states.json


In [48]:
m.choropleth(geo_data=state_geo,
 name='choropleth',
 data=state_data,
 columns=['State', 'Unemployment'],
 key_on='feature.id',
 fill_color='YlGn',
 fill_opacity=0.7,
 line_opacity=0.2)
#folium.LayerControl().add_to(m)
#m.save('#292_folium_chloropleth_USA1.html')

JSONDecodeError: Expecting value: line 7 column 1 (char 6)